# Fan car rover simulation

![sim_fan_car_gazebo.png](images/sim_fan_car_gazebo.png)

This notebook defines SDF for a fan-can rover and spawns the model into Gazebo for simulation with ArduPilot using the SITL-JSON interface. A fan car is a simple rover comprising a flat base on four castor wheels with a steerable fan providing vectored-thrust. 

There are two main interfaces when using `pcg_gazebo`. The higher level interface uses `simulation` objects to define models and other Gazebo objects which are then serialised to XML (SDF format) and passed to Gazebo using a `gazebo_ros_pkgs` service call. The lower level approach works with the SDF directly. We use the latter in order to have the most control over the definition of various physics settings and also because we require custom plugin elements which are not available in the higher level interface. This makes common activities such as defining links and joints more verbose, but allows complete control over the resulting model.

### Dependencies:

#### Python

##### [`pcg_gazebo`](https://github.com/boschresearch/pcg_gazebo)

Two patches are required to `pcb_gazebo`. The first is to support the ArduPilot plugin which has repeated `<control>` elements, the second allows optional elements in the joint physics.    
- https://github.com/boschresearch/pcg_gazebo/pull/141
- https://github.com/boschresearch/pcg_gazebo/pull/142

While these PR's are being reviewed you can use [this fork of `pcb_gazebo` for ArduPilot:](https://github.com/srmainwaring/pcg_gazebo/tree/feature/ardupilot)
    
```bash
# install an editable version of for pcg_gazebo debugging
$ git clone -b feature/ardupilot https://github.com/srmainwaring/pcg_gazebo.git && cd pcg_gazebo
$ python3 -m pip install -e .
```
          
##### [`python-fcl`](https://github.com/DmitryNeverov/python-fcl)

`pcg_gazebo` requires the Python interface to the Flexible Collision Library. We use Dimity Neverov's fork of the BerkeleyAutomation fork which has support for Python3.
    
```bash
# install the master branch of Dmitry Neverov's fork of python-fcl
$ git clone https://github.com/DmitryNeverov/python-fcl.git && cd python-fcl
$ python3 -m pip install -U .
```

#### ROS and Gazebo

`pcg_gazebo` uses `gazebo_ros_pkgs` and `rospy` to communicate with Gazebo and spawn models into a running simulation. To use these features requires a desktop installation of ROS (Melodic or Noetic). ROS2 is not currently supported.

In [18]:
import matplotlib.pyplot as plt
import math
import numpy as np
import os
import random
import rospy
import time

from pcg_gazebo.generators import WorldGenerator
from pcg_gazebo.parsers.sdf import create_sdf_element
from pcg_gazebo.parsers.sdf import Material, Plugin
from pcg_gazebo.simulation.physics import ODE, Physics
from pcg_gazebo.simulation.properties.inertial import Inertial
from pcg_gazebo.task_manager import Server

## Kill the simulation

The cell below kills all running tasks including the Gazebo session - return here to exit the session

In [22]:
# Uncomment to kill all tasks
simulation.kill_all_tasks()

2021-04-29 17:37:38,483 | WARNING | task | Task <gazebo> - Process object is invalid
2021-04-29 17:37:38,484 | WARNING | task | Task <roscore> - Process object is invalid


## Customise the world file

We create a temporary world file to customise the physics and scene. The changes below modify the default physics settings, add a ground plane and light (sun), and update the default scene in the empty world to include sky and clouds.

The world file is saved to `$HOME/.gazebo/worlds/fan_car.world`.

In [3]:
# Customise the world
world = create_sdf_element('world')

# Scene
scene = create_sdf_element('scene')
scene.reset(with_optional_elements=True)
world.scene = scene

# Physics 
world.physics.reset(mode='ode', with_optional_elements=True)
world.physics.max_step_size = 0.001
world.physics.ode.constraints.cfm = 0.001
world.physics.ode.constraints.erp = 0.3
world.physics.ode.constraints.contact_surface_layer = 0.005
world.physics.ode.constraints.contact_max_correcting_vel = 10.0

# Models
include_sun = create_sdf_element('include')
include_sun.uri = 'model://sun'
world.add_include('sun', include_sun)

include_ground_plane = create_sdf_element('include')
include_ground_plane.uri = 'model://ground_plane'
world.add_include('ground_plane', include_ground_plane)

# Export to .world file
sdf_world = create_sdf_element('sdf')
sdf_world.world = world

world_filename = os.path.join(os.path.expanduser('~'), '.gazebo/worlds/fan_car.world')
sdf_world.export_xml(filename=world_filename)

# print(world_filename)
# print(sdf_world)

## Ready the simulation environment

In this section we create a task server and a custom Gazebo simulation task that loads the world file defined in the previous section. Under the hood this task also creates a `roscore` process which is required for communication between `pcg_gazebo` and Gazebo.

The default behaviour is for the ROS master and Gazebo master processes are started on randomly selected ports. The GazeboProxy object can be queried to determine the configuration used for the session.

To interact with the ROS and Gazebo servers in a terminal first set the environment variables reported by `gazebo_proxy.ros_config`. For example:

```bash
# check ROS topics
$ ROS_MASTER_URI=http://localhost:18591
$ rostopic list
/clock
/gazebo/link_states
/gazebo/model_states
/gazebo/parameter_descriptions
/gazebo/parameter_updates
/gazebo/performance_metrics
/gazebo/set_link_state
/gazebo/set_model_state
/rosout
/rosout_agg
```

```bash
# check GAZEBO topics
$ GAZEBO_MASTER_URI=http://localhost:25690
$ gz topic -l                             
/gazebo/default/atmosphere
/gazebo/default/diagnostics
/gazebo/default/factory
/gazebo/default/factory/light
...
```

In [4]:
# Start a simulation server and get a simulation (process_manager) named 'default'
server = Server()
server.create_simulation('default')
simulation = server.get_simulation('default')

In [5]:
# Either: Start an instance of the empty.world
# simulation.create_gazebo_empty_world_task()

# Or: Start an instance of loading the custom world file
simulation.create_gazebo_task(
    name='gazebo',
    world=world_filename,
    gui=True,
    physics='ode',
    paused=False,
    required=True)
   
# Run Gazebo
simulation.run_all_tasks()

# Wait until ready
while not simulation.is_task_running('gazebo'):
    time.sleep(1.0)

print('Gazebo ready')

2021-04-29 15:57:35,622 | WARNING | __init__ | Error testing Gazebo server, message=Command '['rostopic', 'list']' returned non-zero exit status 1.
2021-04-29 15:57:36,003 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1.
2021-04-29 15:57:36,404 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1.
2021-04-29 15:57:36,797 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1.
Gazebo ready


In [6]:
# Create a Gazebo proxy
gazebo_proxy = simulation.get_gazebo_proxy()
print('ROS configuration: {}'.format(gazebo_proxy.ros_config))

ROS configuration: ROS_MASTER_URI=http://localhost:18860, GAZEBO_MASTER_URI=http://localhost:25152


## Create the fan car model

Create the links, joints and plugins for the fan-car rover:

- 1x base link: a base made from a plywood board
- 4x castor wheel links: composite assemblies comprising two links and revolute joints
- 1x fan assembly, a steerable support with a motor and proeller attached
- 2x libLiftDragPlugin.so plugin elements to provide thrust from the propeller
- 1x libArduPilotPlugin.so plugin element to provide ArduPilot SITL-JSON integration 

In [7]:
# Model parameters

# base dimensions
base_length = 0.4
base_width  = 0.3
base_height  = 0.005
base_mass   = 1.25

# castor wheel dimensions
steer_length = 0.012
steer_radius = 0.01
steer_mass = 0.05

steer_offset_x = -0.05
steer_offset_y = -0.05
steer_offset_z = -0.0

wheel_width  = 0.02
wheel_radius = 0.025
wheel_mass = 0.050
wheel_trail = 0.015

wheel_offset_x = -0.045
wheel_offset_y = -0.045
wheel_offset_z = -0.030

# motor unit dimensions
servo_effort_max = 50.0
servo_velocity_max = 5.0

# motor support dimensions
motor_support_length=0.1
motor_support_radius=0.02
motor_support_mass=0.1
motor_support_offset=0.05
motor_support_joint_lower_deg=-180
motor_support_joint_upper_deg=180

# motor dimensions
motor_length=0.04
motor_radius=0.02
motor_mass=0.06
motor_axle_length=0.065
motor_axle_radius=0.0025
motor_axle_mass=0.01
motor_axle_offset=0.0075

# propeller dimensions
prop_length=0.2
prop_radius=0.005
prop_mass=0.01
prop_offset=0.01

# propeller centre of pressure visuals
prop_cp_radius=0.0075
prop_cp_offset=0.05

### base link

The rover base is a flat board

In [8]:
# Create base link
def create_base_link(
    name,
    base_length,
    base_width,
    base_height,
    base_mass,
    ):

    base_link = create_sdf_element('link')
    base_link.name = name + '_link'
    base_link.add_collision('collision')
    base_link.add_visual('visual')

    # add inertial
    inertial = Inertial.create_inertia(
        'cuboid',
        mass=base_mass,
        length_x=base_length,
        length_y=base_width,
        length_z=base_height).to_sdf()
    base_link.inertial = inertial

    # set collision
    collision = base_link.get_collision_by_name('collision')
    collision.geometry.box = create_sdf_element('box')
    collision.geometry.box.size = [base_length, base_width, base_height]    
    collision.surface = create_sdf_element('surface')
    collision.surface.friction = create_sdf_element('friction')
    collision.surface.friction.reset(mode='surface')
    collision.surface.friction.ode = create_sdf_element('ode')    
    collision.surface.friction.ode.reset(mode='collision')
    collision.surface.friction.ode.rm_child('fdir1')
    collision.surface.friction.ode.rm_child('slip1')
    collision.surface.friction.ode.rm_child('slip2')
    collision.surface.friction.ode.mu = 1.0
    collision.surface.friction.ode.mu2 = 1.0
    
    # set visual
    visual = base_link.get_visual_by_name('visual')
    visual.geometry.box = collision.geometry.box
    visual.material = Material.get_gazebo_material('Gazebo/Red')
    
    return base_link

base_link = create_base_link(
    name='base',
    base_length=base_length,
    base_width=base_width,
    base_height=base_height,
    base_mass=base_mass)

# print(base_link)

### joint physics

There is an issue with the ode settor on the physics element (the mode is not set correctly), so you cannot use the following to only set the limit:

```python
physics = create_sdf_element('physics')
physics.reset(mode='joint')
physics.ode = create_sdf_element('ode')
physics.ode.reset(mode='joint')
physics.ode.limit.cfm = 0.0
physics.ode.limit.erp = 0.2
```

The work-around is to remove unwanted elements

```python
physics = create_sdf_element('physics')
physics.reset(mode='joint', with_optional_elements=True)
physics.rm_child('provide_feedback')
physics.rm_child('simbody')
physics.ode.rm_child('provide_feedback')
physics.ode.rm_child('cfm_damping')
physics.ode.rm_child('cfm')
physics.ode.rm_child('erp')
physics.ode.limit.cfm = 0.0
physics.ode.limit.erp = 0.2
```

### castor wheel links

Each of the four castor wheels comprises two link elements and two revolute joints.

The wheel is free to rotate about a horizontal axle (revolute joint) and is connected to a vertical steering link which is offset from the wheel axle (the wheel trail). The steering link is connected to the base by another revolute joint aligned to the z-axis. Both joints are free to rotate about their single axis without limit.

In [9]:
# Create castor wheel links
def create_castor_wheel(
    name,
    lat_reflect,
    lon_reflect,
    base_link,
    base_length,
    base_width,
    steer_length,
    steer_radius,
    steer_mass,
    steer_offset_x,
    steer_offset_y,
    steer_offset_z,
    wheel_width,
    wheel_radius,
    wheel_mass,
    wheel_trail
    ):

    # physics
    castor_damping = 0.0001
    castor_friction = 0.0001
    wheel_damping = 0.0001
    wheel_friction = 0.03
    
    # castor link (steering axis)
    castor_link = create_sdf_element('link')
    castor_link.name = name + '_castor_link'    
    castor_link.pose = [
        (base_length/2 + steer_offset_x) * lon_reflect,
        (base_width/2 + steer_offset_y) * lat_reflect,
        steer_offset_z,
        0, 0 ,0]
    castor_link.add_collision('collision')
    castor_link.add_visual('visual')
    
    # add inertial   
    inertial = Inertial.create_inertia(
        'solid_cylinder',
        mass=steer_mass,
        radius=steer_radius,
        length=steer_length,
        axis=[0, 0, 1]).to_sdf()
    castor_link.inertial = inertial
    
    # set collision
    collision = castor_link.get_collision_by_name('collision')
    collision.geometry.cylinder = create_sdf_element('cylinder')
    collision.geometry.cylinder.radius = steer_radius
    collision.geometry.cylinder.length = steer_length
    collision.surface = create_sdf_element('surface')
    collision.surface.friction = create_sdf_element('friction')
    collision.surface.friction.reset(mode='surface')
    collision.surface.friction.ode = create_sdf_element('ode')    
    collision.surface.friction.ode.reset(mode='collision')
    collision.surface.friction.ode.rm_child('fdir1')
    collision.surface.friction.ode.rm_child('slip1')
    collision.surface.friction.ode.rm_child('slip2')
    collision.surface.friction.ode.mu = 1.0
    collision.surface.friction.ode.mu2 = 1.0
#     print(collision)

    # set visual
    visual = castor_link.get_visual_by_name('visual')
    visual.geometry.cylinder = collision.geometry.cylinder
    visual.material = Material.get_gazebo_material('Gazebo/Yellow')
    
    # castor joint (to base_link)
    castor_joint = create_sdf_element('joint')
    castor_joint.name = name + '_castor_joint'
    castor_joint.parent = base_link.name
    castor_joint.child = castor_link.name
    castor_joint.axis = create_sdf_element('axis')
    castor_joint.axis.xyz = [0, 0, 1]
    castor_joint.axis.dynamics = create_sdf_element('dynamics')
    castor_joint.axis.dynamics.damping = castor_damping
    castor_joint.axis.dynamics.friction = castor_friction

    # customise joint physics
    castor_joint.physics = create_sdf_element('physics')
    castor_joint.physics.reset(mode='joint', with_optional_elements=True)
    castor_joint.physics.rm_child('provide_feedback')
    castor_joint.physics.rm_child('simbody')
    castor_joint.physics.ode.rm_child('provide_feedback')
    castor_joint.physics.ode.rm_child('cfm_damping')
    castor_joint.physics.ode.rm_child('cfm')
    castor_joint.physics.ode.rm_child('erp')
    castor_joint.physics.ode.limit.cfm = 0.0
    castor_joint.physics.ode.limit.erp = 0.2
#     print(castor_joint)

    # wheel link (steering axis)
    wheel_link = create_sdf_element('link')
    wheel_link.name = name + '_wheel_link'    
    wheel_link.pose = [
        (base_length/2 + steer_offset_x) * lon_reflect + wheel_trail * -1.0,
        (base_width/2 + steer_offset_y) * lat_reflect,
        steer_offset_z + (steer_length + wheel_radius) * -1.0,
        -1.0 * math.pi/2, 0 ,0]
    wheel_link.add_collision('collision')
    wheel_link.add_visual('visual')
    
    # add inertial   
    inertial = Inertial.create_inertia(
        'solid_cylinder',
        mass=wheel_mass,
        radius=wheel_radius,
        length=wheel_width,
        axis=[0, 0, 1]).to_sdf()
    wheel_link.inertial = inertial
    
    # set collision
    collision = wheel_link.get_collision_by_name('collision')
    collision.geometry.cylinder = create_sdf_element('cylinder')
    collision.geometry.cylinder.radius = wheel_radius
    collision.geometry.cylinder.length = wheel_width
    collision.surface = create_sdf_element('surface')
    collision.surface.friction = create_sdf_element('friction')
    collision.surface.friction.reset(mode='surface')
    collision.surface.friction.ode = create_sdf_element('ode')    
    collision.surface.friction.ode.reset(mode='collision')
    collision.surface.friction.ode.rm_child('fdir1')
    collision.surface.friction.ode.rm_child('slip1')
    collision.surface.friction.ode.rm_child('slip2')
    collision.surface.friction.ode.mu = 1.0
    collision.surface.friction.ode.mu2 = 1.0
#     print(collision)
    
    # set visual
    visual = wheel_link.get_visual_by_name('visual')
    visual.geometry.cylinder = collision.geometry.cylinder
    visual.material = Material.get_gazebo_material('Gazebo/Green')
    
    # wheel joint (to base_link)
    wheel_joint = create_sdf_element('joint')
    wheel_joint.name = name + '_wheel_joint'
    wheel_joint.parent = castor_link.name
    wheel_joint.child = wheel_link.name
    wheel_joint.axis = create_sdf_element('axis')
    wheel_joint.axis.xyz = [0, 0, 1]  
    wheel_joint.axis.dynamics = create_sdf_element('dynamics')
    wheel_joint.axis.dynamics.damping = wheel_damping
    wheel_joint.axis.dynamics.friction = wheel_friction
            
    # customise joint physics
    wheel_joint.physics = create_sdf_element('physics')
    wheel_joint.physics.reset(mode='joint', with_optional_elements=True)
    wheel_joint.physics.rm_child('provide_feedback')
    wheel_joint.physics.rm_child('simbody')
    wheel_joint.physics.ode.rm_child('provide_feedback')
    wheel_joint.physics.ode.rm_child('cfm_damping')
    wheel_joint.physics.ode.rm_child('cfm')
    wheel_joint.physics.ode.rm_child('erp')
    wheel_joint.physics.ode.limit.cfm = 0.0
    wheel_joint.physics.ode.limit.erp = 0.2
            
    return castor_link, castor_joint, wheel_link, wheel_joint

# wheel links
front_left_castor_link, front_left_castor_joint, \
front_left_wheel_link, front_left_wheel_joint = create_castor_wheel(
    name='front_left',
    lat_reflect=1,
    lon_reflect=1,
    base_link=base_link,
    base_length=base_length,
    base_width=base_width,
    steer_length=steer_length,
    steer_radius=steer_radius,
    steer_mass=steer_mass,
    steer_offset_x=steer_offset_x,
    steer_offset_y=steer_offset_y,
    steer_offset_z=steer_offset_z,
    wheel_width=wheel_width,
    wheel_radius=wheel_radius,
    wheel_mass=wheel_mass,
    wheel_trail=wheel_trail)

front_right_castor_link, front_right_castor_joint, \
front_right_wheel_link, front_right_wheel_joint = create_castor_wheel(
    name='front_right',
    lat_reflect=-1,
    lon_reflect=1,
    base_link=base_link,
    base_length=base_length,
    base_width=base_width,
    steer_length=steer_length,
    steer_radius=steer_radius,
    steer_mass=steer_mass,
    steer_offset_x=steer_offset_x,
    steer_offset_y=steer_offset_y,
    steer_offset_z=steer_offset_z,
    wheel_width=wheel_width,
    wheel_radius=wheel_radius,
    wheel_mass=wheel_mass,
    wheel_trail=wheel_trail)

back_left_castor_link, back_left_castor_joint, \
back_left_wheel_link, back_left_wheel_joint = create_castor_wheel(
    name='back_left',
    lat_reflect=1,
    lon_reflect=-1,
    base_link=base_link,
    base_length=base_length,
    base_width=base_width,
    steer_length=steer_length,
    steer_radius=steer_radius,
    steer_mass=steer_mass,
    steer_offset_x=steer_offset_x,
    steer_offset_y=steer_offset_y,
    steer_offset_z=steer_offset_z,
    wheel_width=wheel_width,
    wheel_radius=wheel_radius,
    wheel_mass=wheel_mass,
    wheel_trail=wheel_trail)

back_right_castor_link, back_right_castor_joint, \
back_right_wheel_link, back_right_wheel_joint = create_castor_wheel(
    name='back_right',
    lat_reflect=-1,
    lon_reflect=-1,
    base_link=base_link,
    base_length=base_length,
    base_width=base_width,
    steer_length=steer_length,
    steer_radius=steer_radius,
    steer_mass=steer_mass,
    steer_offset_x=steer_offset_x,
    steer_offset_y=steer_offset_y,
    steer_offset_z=steer_offset_z,
    wheel_width=wheel_width,
    wheel_radius=wheel_radius,
    wheel_mass=wheel_mass,
    wheel_trail=wheel_trail)

# print(front_left_castor_link)
# print(front_left_castor_joint)
# print(front_left_wheel_link)
# print(front_left_wheel_joint)

### fan assembly

The fan assembly consists of a steerable support tower and motor with an attached propeller. The model is stylised and uses a cylinder to represent the support tower and motor with the propeller represented using a mesh visual.

Thrust is generated when the motor joint rotates via two Gazebo LiftDragPlugin elements attached to the motor and the centre of pressure of each propeller blade.

In [10]:
# Create fan assembly
def create_steer_rotor(
    name,
    base_link,
    base_length,
    base_width,
    support_radius,
    support_length,
    support_mass,
    support_offset,
    support_joint_lower_deg,
    support_joint_upper_deg,
    support_servo_effort_max,
    support_servo_velocity_max,
    motor_length,
    motor_radius,
    motor_mass,
    motor_axle_length,
    motor_axle_radius,
    motor_axle_offset,
    prop_length,
    prop_radius,
    prop_mass,
    prop_offset,
    prop_cp_radius,
    prop_cp_offset    
    ):
    
    # physics
    steer_rotor_damping = 0.0001
    steer_rotor_friction = 0.0001
    motor_damping = 0.0001
    motor_friction = 0.0001
    
    # steer rotor link
    steer_rotor_link = create_sdf_element('link')
    steer_rotor_link.name = name + '_steer_rotor_link'    
    steer_rotor_link.pose = [
        (base_length/2 * -1.0 + support_offset),
        0,
        (support_length + base_height)/2,
        0, 0 ,0]
    steer_rotor_link.add_collision('collision')
    steer_rotor_link.add_visual('visual')
    
    # add inertial   
    inertial = Inertial.create_inertia(
        'solid_cylinder',
        mass=support_mass,
        radius=support_radius,
        length=support_length,
        axis=[0, 0, 1]).to_sdf()
    steer_rotor_link.inertial = inertial
    
    # set collision
    collision = steer_rotor_link.get_collision_by_name('collision')
    collision.geometry.cylinder = create_sdf_element('cylinder')
    collision.geometry.cylinder.radius = support_radius
    collision.geometry.cylinder.length = support_length
    collision.surface = create_sdf_element('surface')
    collision.surface.friction = create_sdf_element('friction')
    collision.surface.friction.reset(mode='surface')
    collision.surface.friction.ode = create_sdf_element('ode')    
    collision.surface.friction.ode.reset(mode='collision')
    collision.surface.friction.ode.rm_child('fdir1')
    collision.surface.friction.ode.rm_child('slip1')
    collision.surface.friction.ode.rm_child('slip2')
    collision.surface.friction.ode.mu = 1.0
    collision.surface.friction.ode.mu2 = 1.0
    
    # set visual
    visual = steer_rotor_link.get_visual_by_name('visual')
    visual.geometry.cylinder = collision.geometry.cylinder
    visual.material = Material.get_gazebo_material('Gazebo/Orange')
    
    # steer rotor joint (to base_link)
    steer_rotor_joint = create_sdf_element('joint')
    steer_rotor_joint.name = name + '_steer_rotor_joint'
    steer_rotor_joint.parent = base_link.name
    steer_rotor_joint.child = steer_rotor_link.name
    steer_rotor_joint.axis = create_sdf_element('axis')
    steer_rotor_joint.axis.xyz = [0, 0, 1]
    steer_rotor_joint.axis.dynamics = create_sdf_element('dynamics')
    steer_rotor_joint.axis.dynamics.damping = steer_rotor_damping
    steer_rotor_joint.axis.dynamics.friction = steer_rotor_friction
    steer_rotor_joint.axis.limit = create_sdf_element('limit')
    steer_rotor_joint.axis.limit.lower = np.radians(motor_support_joint_lower_deg)
    steer_rotor_joint.axis.limit.upper = np.radians(motor_support_joint_upper_deg)
    steer_rotor_joint.axis.limit.effort = servo_effort_max
    steer_rotor_joint.axis.limit.velocity = servo_velocity_max
    
    # customise joint physics
    steer_rotor_joint.physics = create_sdf_element('physics')
    steer_rotor_joint.physics.reset(mode='joint', with_optional_elements=True)
    steer_rotor_joint.physics.rm_child('provide_feedback')
    steer_rotor_joint.physics.rm_child('simbody')
    steer_rotor_joint.physics.ode.rm_child('provide_feedback')
    steer_rotor_joint.physics.ode.rm_child('cfm_damping')
    steer_rotor_joint.physics.ode.rm_child('cfm')
    steer_rotor_joint.physics.ode.rm_child('erp')
    steer_rotor_joint.physics.ode.limit.cfm = 0.0
    steer_rotor_joint.physics.ode.limit.erp = 0.2

    # motor link
    motor_link = create_sdf_element('link')
    motor_link.name = name + '_motor_link'
    motor_link.pose = [
        (base_length/2 * -1.0 + support_offset),
        0,
        support_length + base_height/2 + motor_radius,
        0, math.pi/2 * -1.0, 0]
    motor_link.add_collision('collision')
    motor_link.add_visual('visual')
    motor_link.add_visual('prop_visual')

    # add inertial
    inertial = Inertial.create_inertia(
        'solid_cylinder',
        mass=motor_mass,
        radius=motor_radius,
        length=motor_length,
        axis=[0, 0, 1]).to_sdf()
    motor_link.inertial = inertial

    # set collision
    collision = motor_link.get_collision_by_name('collision')
    collision.geometry.cylinder = create_sdf_element('cylinder')
    collision.geometry.cylinder.radius = motor_radius
    collision.geometry.cylinder.length = motor_length
    collision.surface = create_sdf_element('surface')
    collision.surface.friction = create_sdf_element('friction')
    collision.surface.friction.reset(mode='surface')
    collision.surface.friction.ode = create_sdf_element('ode')    
    collision.surface.friction.ode.reset(mode='collision')
    collision.surface.friction.ode.rm_child('fdir1')
    collision.surface.friction.ode.rm_child('slip1')
    collision.surface.friction.ode.rm_child('slip2')
    collision.surface.friction.ode.mu = 1.0
    collision.surface.friction.ode.mu2 = 1.0

    # set visual
    visual = motor_link.get_visual_by_name('visual')
    visual.geometry.cylinder = collision.geometry.cylinder    
    visual.material = Material.get_gazebo_material('Gazebo/Orange')

    # set prop visual
    prop_visual = motor_link.get_visual_by_name('prop_visual')
    # the visual's pose is defined relative to the link     
    prop_visual.pose = [
        0, 0, motor_length/2,
        0, 0, 0]
    mesh = create_sdf_element('mesh')
    mesh.uri = '/Users/rhys/CloudStation/Projects/Notebooks/gazebo/meshes/propeller_9x4.7.stl'
    prop_visual.geometry.mesh = mesh
    prop_visual.material = Material.get_gazebo_material('Gazebo/Gray')

    # add motor joint
    motor_joint = create_sdf_element('joint')
    motor_joint.name = name + '_motor_joint'
    motor_joint.parent = steer_rotor_link.name
    motor_joint.child = motor_link.name
    motor_joint.axis = create_sdf_element('axis')
    motor_joint.axis.dynamics = create_sdf_element('dynamics')
    motor_joint.axis.dynamics.damping = motor_damping
    motor_joint.axis.dynamics.friction = motor_friction
    
    # add sensor to joint
    motor_joint.sensor = create_sdf_element('sensor')
    motor_joint.sensor.reset(mode='force_torque')
    motor_joint.sensor.name = 'force_torque_sensor'
    motor_joint.sensor.type = 'force_torque'
    motor_joint.sensor.always_on = True
    motor_joint.sensor.update_rate = 30
    motor_joint.sensor.visualize = True

    # customise joint physics
    motor_joint.physics = create_sdf_element('physics')
    motor_joint.physics.reset(mode='joint', with_optional_elements=True)
    motor_joint.physics.rm_child('provide_feedback')
    motor_joint.physics.rm_child('simbody')
    motor_joint.physics.ode.rm_child('provide_feedback')
    motor_joint.physics.ode.rm_child('cfm_damping')
    motor_joint.physics.ode.rm_child('cfm')
    motor_joint.physics.ode.rm_child('erp')
    motor_joint.physics.ode.limit.cfm = 0.0
    motor_joint.physics.ode.limit.erp = 0.2
    
    return steer_rotor_link, steer_rotor_joint, motor_link, motor_joint

steer_rotor_link, steer_rotor_joint, \
motor_link, motor_joint = create_steer_rotor(
    name='back',
    base_link=base_link,
    base_length=base_length,
    base_width=base_width,
    support_radius=motor_support_radius,
    support_length=motor_support_length,
    support_mass=motor_support_mass,
    support_offset=motor_support_offset,
    support_joint_lower_deg=motor_support_joint_lower_deg,
    support_joint_upper_deg=motor_support_joint_upper_deg,
    support_servo_effort_max=servo_effort_max,
    support_servo_velocity_max=servo_velocity_max,
    motor_length=motor_length,
    motor_radius=motor_radius,
    motor_mass=motor_mass,
    motor_axle_length=motor_axle_length,
    motor_axle_radius=motor_axle_radius,
    motor_axle_offset=motor_axle_offset,
    prop_length=prop_length,
    prop_radius=prop_radius,
    prop_mass=prop_mass,
    prop_offset=prop_offset,
    prop_cp_radius=prop_cp_radius,
    prop_cp_offset=prop_cp_offset)

# print(steer_rotor_link)
# print(steer_rotor_joint)
# print(motor_link)
# print(motor_joint)

### IMU sensor

The IMU sensor is attached to the base link via a revolute joint with hard limits set to constrain movement. This may seem like an odd arrangement but it results in a more stable simulation than using a 'fixed' joint. If the IMU link is not stable then significant vibrations will propagate to the flight controller and ruin the EKF.

Ideally the IMU sensor would be embedded in the base link of the model and not be on a low mass link of its own. The reason for placing it on a separate link is that this is a general approach that also works when adapting existing models for use with ArduPilot. 

In [11]:
# add imu link (required by the ardupilot plugin)
def create_imu(
    base_link
    ):
    imu_link = create_sdf_element('link')
    imu_link.name = 'imu_link'
    imu_link.add_sensor('imu_sensor')

    # add inertial
    imu_inertial = Inertial.create_inertia(
        'cuboid',
        mass=0.01,
        length_x=0.01,
        length_y=0.01,
        length_z=0.01).to_sdf()
    imu_link.inertial = imu_inertial

    # add imu sensor
    imu_sensor = imu_link.get_sensor_by_name('imu_sensor')
    imu_sensor.reset(mode='imu', with_optional_elements=False)
    imu_sensor.name = 'imu_sensor'
    imu_sensor.type = 'imu'
    imu_sensor.pose = [0, 0, 0, math.pi, 0, 0]
    imu_sensor.always_on = 1
    imu_sensor.update_rate = 500

    # add imu joint
    imu_joint = create_sdf_element('joint')
    imu_joint.name = 'imu_joint'
    imu_joint.parent = base_link.name
    imu_joint.child = imu_link.name
    imu_joint.axis = create_sdf_element('axis')
    imu_joint.axis.limit.lower = 0
    imu_joint.axis.limit.upper = 0

    return imu_link, imu_joint, imu_sensor

imu_link, imu_joint, imu_sensor = create_imu(base_link=base_link)
# print(imu_link)
# print(imu_joint)
# print('{}::{}'.format(imu_link.name, imu_sensor.name))

### libLiftDragPlugin.so

Two LiftDragPlugin elements are specified for the propeller. The parameters have been adapted from the Iris model published in the `ardupilot_gazebo` repo. In this setup the plugin is configured to have zero pitch and uses the lift coefficient corresponding to zero angle of attack. This is not quite right, but it turns out that the lift generated by this arrangement and the provided coefficients roughly corresponds to the expected thrust from a 8045 propeller spinning at around 800 rad/s.

```xml
<plugin name="prop_blade_1_liftdrag" filename="libLiftDragPlugin.so">
  <a0>0.3</a0>
  <alpha_stall>1.4</alpha_stall>
  <cla>4.25</cla>
  <cda>0.1</cda>
  <cma>0</cma>
  <cla_stall>-0.025</cla_stall>
  <cda_stall>0</cda_stall>
  <cma_stall>0</cma_stall>
  <area>0.002</area>
  <air_density>1.2041</air_density>
  <cp>0.084 0 0</cp>
  <forward>0 1.0 0</forward>
  <upward>0 0 1</upward>
  <link_name>back_motor_link</link_name>
</plugin>
```

In [12]:
# add lift-drag plugin for prop
def create_prop_blade_plugin_args(direction):
    plugin_args = dict(
        a0=0.3,
        alpha_stall=1.4,
        cla=4.25,
        cda=0.1,
        cma=0,
        cla_stall=-0.025,
        cda_stall=0,
        cma_stall=0,
        area=0.002,
        air_density=1.2041,
        cp=[0.084 * direction, 0, 0],
        forward=[0, 1 * direction, 0],
        upward=[0, 0, 1],
        link_name=motor_link.name,
    )
    return plugin_args

prop_blade_1_plugin = create_sdf_element('plugin')
prop_blade_1_plugin.name = 'prop_blade_1_liftdrag'
prop_blade_1_plugin.filename = 'libLiftDragPlugin.so'
prop_blade_1_plugin.value = create_prop_blade_plugin_args(direction=1.0)

prop_blade_2_plugin = create_sdf_element('plugin')
prop_blade_2_plugin.name = 'prop_blade_2_liftdrag'
prop_blade_2_plugin.filename = 'libLiftDragPlugin.so'
prop_blade_2_plugin.value = create_prop_blade_plugin_args(direction=-1.0)

# print(prop_blade_1_plugin)
# print(prop_blade_2_plugin)

### ardupilot_plugin

These are the settings for the ArduPilotPlugin for a Rover vehicle type using `SERVO1_FUNCTION = 26` (Ground steering) and `SERVO3_FUNCTION = 70` (Throttle). The PID gains for the motor velocity are fairly sensitive. They need to be large enough to ensure a good response without introducing oscillations that will cause excessive vibrations. 

```xml
<plugin name="ardupilot_plugin" filename="libArduPilotPlugin.so">
  <fdm_addr>127.0.0.1</fdm_addr>
  <fdm_port_in>9002</fdm_port_in>
  <connectionTimeoutMaxCount>5</connectionTimeoutMaxCount>
  <modelXYZToAirplaneXForwardZDown>0 0 0 3.141592653589793 0 0</modelXYZToAirplaneXForwardZDown>
  <gazeboXYZToNED>0 0 0 3.141592653589793 0 0</gazeboXYZToNED>
  <imuName>imu_link::imu_sensor</imuName>
  <control channel="0">
    <jointName>back_steer_rotor_joint</jointName>
    <useForce>1</useForce>
    <multiplier>3.141592653</multiplier>
    <offset>-0.5</offset>
    <servo_min>1000</servo_min>
    <servo_max>2000</servo_max>
    <type>POSITION</type>
    <p_gain>10</p_gain>
    <i_gain>0.1</i_gain>
    <d_gain>0.02</d_gain>
    <i_max>1</i_max>
    <i_min>-1</i_min>
    <cmd_max>100.0</cmd_max>
    <cmd_min>-100.0</cmd_min>
  </control>
  <control channel="2">
    <jointName>back_motor_joint</jointName>
    <useForce>1</useForce>
    <multiplier>-800</multiplier>
    <offset>0</offset>
    <servo_min>1000</servo_min>
    <servo_max>2000</servo_max>
    <type>VELOCITY</type>
    <p_gain>0.007</p_gain>
    <i_gain>0.005</i_gain>
    <d_gain>0</d_gain>
    <i_max>0.1</i_max>
    <i_min>-0.1</i_min>
    <cmd_max>10.0</cmd_max>
    <cmd_min>-10.0</cmd_min>
  </control>
</plugin>
```

In [13]:
# add ardupilot plugin
ardupilot_plugin_args = dict(
    fdm_addr='127.0.0.1',
    fdm_port_in=9002,
    connectionTimeoutMaxCount=5,
    modelXYZToAirplaneXForwardZDown=[0, 0, 0, math.pi, 0, 0],
    gazeboXYZToNED=[0, 0, 0, math.pi, 0, 0],
    imuName='{}::{}'.format(imu_link.name, imu_sensor.name),
    control=list([
        dict(
            attributes=dict(
                channel='0'
            ),
            value=dict(
                jointName=steer_rotor_joint.name,
                useForce=1,
                multiplier=3.141592653,
                offset=-0.5,
                servo_min=1000,
                servo_max=2000,
                type='POSITION',
                p_gain=10,
                i_gain=0.1,
                d_gain=0.02,
                i_max=1,
                i_min=-1,
                cmd_max=100.0,
                cmd_min=-100.0
            )
        ),
        dict(
            attributes=dict(
                channel='2'
            ),
            value=dict(
                jointName=motor_joint.name,
                useForce=1,
                multiplier=-800,
                offset=0,
                servo_min=1000,
                servo_max=2000,
                type='VELOCITY',
                p_gain=0.007,
                i_gain=0.005,
                d_gain=0,
                i_max=0.1,
                i_min=-0.1,
                cmd_max=10.0,
                cmd_min=-10.0
            )
        )]
    )
)

ardupilot_plugin = create_sdf_element('plugin')
ardupilot_plugin.name = 'ardupilot_plugin'
ardupilot_plugin.filename = 'libArduPilotPlugin.so'
ardupilot_plugin.value = ardupilot_plugin_args
# print(ardupilot_plugin)

### Create the model

The links, joints and plugin elements are composed to form the final model.

In [14]:
# Create model element
model = create_sdf_element('model')
model.name = 'fan_car'

# add links
model.add_link(base_link.name, base_link)
model.add_link(front_left_castor_link.name, front_left_castor_link)
model.add_link(front_right_castor_link.name, front_right_castor_link)
model.add_link(back_left_castor_link.name, back_left_castor_link)
model.add_link(back_right_castor_link.name, back_right_castor_link)
model.add_link(front_left_wheel_link.name, front_left_wheel_link)
model.add_link(front_right_wheel_link.name, front_right_wheel_link)
model.add_link(back_left_wheel_link.name, back_left_wheel_link)
model.add_link(back_right_wheel_link.name, back_right_wheel_link)
model.add_link(steer_rotor_link.name, steer_rotor_link)
model.add_link(motor_link.name, motor_link)
model.add_link(imu_link.name, imu_link)

# add joints
model.add_joint(front_left_castor_joint.name, front_left_castor_joint)
model.add_joint(front_right_castor_joint.name, front_right_castor_joint)
model.add_joint(back_left_castor_joint.name, back_left_castor_joint)
model.add_joint(back_right_castor_joint.name, back_right_castor_joint)
model.add_joint(front_left_wheel_joint.name, front_left_wheel_joint)
model.add_joint(front_right_wheel_joint.name, front_right_wheel_joint)
model.add_joint(back_left_wheel_joint.name, back_left_wheel_joint)
model.add_joint(back_right_wheel_joint.name, back_right_wheel_joint)
model.add_joint(steer_rotor_joint.name, steer_rotor_joint)
model.add_joint(motor_joint.name, motor_joint)
model.add_joint(imu_joint.name, imu_joint)

# add plugins
model.add_plugin(prop_blade_1_plugin.name, prop_blade_1_plugin)
model.add_plugin(prop_blade_2_plugin.name, prop_blade_2_plugin)
model.add_plugin(ardupilot_plugin.name, ardupilot_plugin)

sdf_model = create_sdf_element('sdf')
sdf_model.reset(mode='model')
sdf_model.add_model(model=model)

# print(sdf_model)

In [15]:
# Export the model (optional)
model_filename = os.path.join(os.path.expanduser('~'), \
    'Code/ardupilot/simulation/ardupilot_gazebo_models/config/fan_car/fan_car_2.sdf')
# sdf_model.export_xml(filename=model_filename)

## Spawn the SDF model in Gazebo

Here we spawn a single model into the running Gazebo session. The model can be updated and reset without restarting the simulation by recalculating the cell below.


Run the model with SITL-JSON using:

```bash
sim_vehicle.py -v Rover -f JSON --custom-location="51.5716035,-4.0302578,0,10.0" --aircraft=fan_car --mavproxy-args "--out=udp:192.168.1.83:14550" --console --map
```

The parameters for the model are described here: [Fan Car - a Vectored-Thrust Rover](https://discuss.ardupilot.org/t/fan-car-a-vectored-thrust-rover/70036)


In [16]:
# Reset models if they exist
for model_name in gazebo_proxy.get_model_names():
    if 'fan_car' in model_name:
        gazebo_proxy.delete_model(model_name)

time.sleep(1)

# Spawn sdf model    
gazebo_proxy.spawn_sdf_model(
    robot_namespace=model.name,
    xml=sdf_model.to_xml_as_str(),
    pos=[0, 0, base_height/2 + 0.01],
    rot=[0, 0, 0],
    reference_frame='world')

True

In [17]:
# There is an issue in macOS where the sceneviewer causes the notebook to become unresponsive
# from pcg_gazebo.simulation import SimulationModel
# sim_model = SimulationModel.from_sdf(model)
# sim_model.show()